In [1]:
from flask import Flask,render_template,request
import pickle
import numpy as np

In [2]:
popular_df = pickle.load(open('popular.pkl','rb'))
pt = pickle.load(open('pt.pkl','rb'))
books = pickle.load(open('books.pkl','rb'))
similarity_scores = pickle.load(open('similarity_scores.pkl','rb'))

In [3]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html',
                           book_name = list(popular_df['Book-Title'].values),
                           author=list(popular_df['Book-Author'].values),
                           image=list(popular_df['Image-URL-M'].values),
                           votes=list(popular_df['num_ratings'].values),
                           rating=list(popular_df['avg_rating'].values)
                           )

In [4]:
@app.route('/recommend')
def recommend_ui():
    return render_template('recommend.html')

In [5]:
@app.route('/recommend_books',methods=['post'])
def recommend():
    user_input = request.form.get('user_input')
    index = np.where(pt.index == user_input)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)

    print(data)

    return render_template('recommend.html',data=data)

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jan/2023 12:40:55] "POST /recommend_books HTTP/1.1" 200 -


[['1984', 'George Orwell', 'http://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg'], ['Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson', 'Louise Rennison', 'http://images.amazon.com/images/P/0064472272.01.MZZZZZZZ.jpg'], ['Midnight', 'Dean R. Koontz', 'http://images.amazon.com/images/P/0425118703.01.MZZZZZZZ.jpg'], ['Second Nature', 'Alice Hoffman', 'http://images.amazon.com/images/P/0399139087.01.MZZZZZZZ.jpg']]


127.0.0.1 - - [03/Jan/2023 12:41:22] "POST /recommend_books HTTP/1.1" 200 -


[['Winter Moon', 'Dean R. Koontz', 'http://images.amazon.com/images/P/0553582933.01.MZZZZZZZ.jpg'], ['Dark Rivers of the Heart', 'Dean R. Koontz', 'http://images.amazon.com/images/P/034539657X.01.MZZZZZZZ.jpg'], ['Shadowfires', 'Dean R. Koontz', 'http://images.amazon.com/images/P/0425136981.01.MZZZZZZZ.jpg'], ['While My Pretty One Sleeps', 'Mary Higgins Clark', 'http://images.amazon.com/images/P/0671673688.01.MZZZZZZZ.jpg']]
